In [ ]:
library(dplyr)
library(Seurat)
library(spacexr)

scRNA=readRDS('/csb2/finishedProject/HuiLiJian_DN_2025/10x_HD/5-final/2-RCTD/scRNA_ref.rds')
hd=readRDS('/csb2/finishedProject/HuiLiJian_DN_2025/10x_HD/5-final/2-RCTD/17s30980&18s18941.016um/17s30980&18s18941.016um.rds')
coords = read.csv('/csb2/finishedProject/HuiLiJian_DN_2025/10x_HD/5-final/2-RCTD/17s30980&18s18941.016um/17s30980&18s18941.016um.coords.csv',row.names=1)
# scRNA <- RenameIdents(scRNA, "NK/T" = "NKT")
    
RCTD_result <- create.RCTD(SpatialRNA(coords,
                                      GetAssayData(hd,assay='RNA',slot='counts'),use_fake_coords=FALSE), 
        Reference(GetAssayData(scRNA,assay='RNA',slot='counts'),Idents(scRNA)), 
        max_cores=20,test_mode=FALSE,MAX_MULTI_TYPES = 2,UMI_min = 0)
RCTD_result@config$max_cores <- 8
RCTD_result <- run.RCTD(RCTD_result,doublet_mode='doublet')
saveRDS(RCTD_result, file = '/csb2/finishedProject/HuiLiJian_DN_2025/10x_HD/5-final/2-RCTD/17s30980&18s18941.016um/RCTD_result.doublet.rds')
# write.csv(RCTD_result@results$weights,'/picb/lihonglab/tangzhixuan/HuiLiJian_DN_2025/RCTD/RCTD_result.csv')

# write.csv(normalize_weights(RCTD_result@results$weights),
#           paste0('/picb/lihonglab/tangzhixuan/HuiLiJian_DN_2025/RCTD/RCTD_normalized_result.csv'))
    


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching SeuratObject

Warning message in Reference(GetAssayData(scRNA, assay = "RNA", slot = "counts"), :
“Reference: number of cells per cell type is 53010, larger than maximum allowable of 10000. Downsampling number of cells to: 10000”
Begin: process_cell_type_info

process_cell_type_info: number of cells in reference: 76120

process_cell_type_info: number of genes in reference: 36601




   Neutrophil           CAF         Tumor          CD4T    Macrophage 
        10000          6789         10000         10000         10000 
  Endothelial          CD8T Smooth muscle        B cell 
        10000         10000          2741          6590 


Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 2.7 GiB”
Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 1.9 GiB”
Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 2.7 GiB”
Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 2.7 GiB”
Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 2.7 GiB”
Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 2.7 GiB”
Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 2.7 GiB”
Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 1.8 GiB”
End: process_cell_type_info

create.RCTD: getting regression differentially expressed genes: 

get_de_genes: Neutrophil found DE genes: 538

get_de_genes: CAF found DE genes: 208

get_de_genes: Tumor found DE genes: 306

get

In [ ]:
library(dplyr)
library(Seurat)
library(spacexr)
rctd_14u = readRDS('/csb2/finishedProject/HuiLiJian_DN_2025/10x_HD/5-final/2-RCTD/17s30980&18s18941.016um/RCTD_result.doublet.rds')
# 2. check the result of doublet mode
rctd_14u_res <- rctd_14u@results

head(rctd_14u_res$results_df)      # 'spot_class' may include 'reject', 'singlet', 'doublet_certain', 'doublet_uncertain'
head(rctd_14u_res$weights_doublet) # RCTD values of doublet mode

## 3. build RCTD matrix of doublet mode
rctd_14u_res_db <- rctd_14u_res$weights_doublet
dim(rctd_14u_res_db)

celltype_firsec <- rctd_14u_res$results_df[, c('first_type', 'second_type')]

if (!is.data.frame(rctd_14u_res_db)) {
    rctd_14u_res_db <- as.data.frame(as.matrix(rctd_14u_res_db))
}

if (!is.data.frame(celltype_firsec)) {
    celltype_firsec <- as.data.frame(celltype_firsec)
}

rctd_14u_res_db <- cbind(rctd_14u_res_db, celltype_firsec[rownames(rctd_14u_res_db), ])
colnames(rctd_14u_res_db) <- c('pct_firstType', 'pct_secondType', 'firstType', 'secondType')
head(rctd_14u_res_db)

rctdPct_fir <- rctd_14u_res_db[, c('pct_firstType', 'firstType')]
rctdPct_fir$bcd <- rownames(rctdPct_fir)  ## add bin barcode
rctdPct_sec <- rctd_14u_res_db[, c('pct_secondType', 'secondType')]
rctdPct_sec$bcd <- rownames(rctdPct_sec)  ## add bin barcode
colnames(rctdPct_fir) <- c('value', 'cellType', 'bcd_14u')
colnames(rctdPct_sec) <- c('value', 'cellType', 'bcd_14u')
rctdPct_mrg <- rbind( rctdPct_fir, rctdPct_sec )
head(rctdPct_mrg)

rctdPct_mrg_mtx <- reshape2::dcast(rctdPct_mrg, bcd_14u ~ cellType, fill = 0)
dim(rctdPct_mrg_mtx)
rctdPct_mrg_mtx[1:5, 1:5]

rownames(rctdPct_mrg_mtx) <- rctdPct_mrg_mtx[, 1]
rctdPct_mrg_mtx <- rctdPct_mrg_mtx[, -1]

write.csv(rctdPct_mrg_mtx,
          paste0('/csb2/finishedProject/HuiLiJian_DN_2025/10x_HD/5-final/2-RCTD/17s30980&18s18941.016um/RCTD_result.doublet.csv'))